In [ ]:
# !sh ./01_make_data_folders.sh

In [ ]:
# !python ./02_make_data_split.py

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
N_JOBS = 15 # cpu_count()
os.environ['MKL_NUM_THREADS'] = str(N_JOBS)
os.environ['OMP_NUM_THREADS'] = str(N_JOBS)

In [ ]:
# DO NOT DO THIS IN YOUR PROJETS
from config import *
from utils import *
from keras_utils import *
from augmentations import *

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.set_random_seed(SEED)
    
%matplotlib inline

In [ ]:
masks_bool = read_pickle_obj('masks_bool.pickle')

In [ ]:
K.clear_session()
model = UEfficientNet(input_shape=(IMG_SIZE,IMG_SIZE,3), dropout_rate=0.25)
model.load_weights(f'./keras.model_640_v3_stage1')

In [ ]:
model.compile(loss=bce_dice_loss, optimizer='adam', metrics=[my_iou_metric])

In [ ]:
epochs = 80
snapshot = SnapshotCallbackBuilder(nb_epochs=epochs,nb_snapshots=1,init_lr=5e-4)
swa = SWA(f'./keras_swa_{TAG}.model', epochs-3)
train_im_path,train_mask_path = f'{DATA_DIR}/keras_im_train',f'{DATA_DIR}/keras_mask_train'

# Generators
training_generator = DataGenerator(
                                   train_im_path=train_im_path,
                                   train_mask_path=train_mask_path,
                                   augmentations=AUGMENTATIONS_TRAIN,
                                   img_size=IMG_SIZE,
                                   debug=False,
                                   stratified=True,
                                   shuffle=True,
                                   masks_bool=masks_bool)

valid_im_path,valid_mask_path = f'{DATA_DIR}/keras_im_val',f'{DATA_DIR}/keras_mask_val'

validation_generator = DataGenerator(train_im_path=valid_im_path,
                                     train_mask_path=valid_mask_path,
                                     augmentations=AUGMENTATIONS_TEST,
                                     img_size=IMG_SIZE,
                                     batch_size=1,
                                     debug=False,
                                     shuffle=False)

history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              use_multiprocessing=False,
                              epochs=epochs,
                              verbose=1,
                              callbacks=snapshot.get_callbacks())

In [ ]:
quit()